## Setting up the imports for the project

In [176]:
import torch
import torch.nn as nn
import pandas as pd
import re
import numpy as np
import unicodedata

In [177]:
# Reading the data, and printing the head
data = pd.read_csv("hindi_english_parallel.csv")
data.head()

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default


In [178]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1561841 entries, 0 to 1561840
Data columns (total 2 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   hindi    1555784 non-null  object
 1   english  1560964 non-null  object
dtypes: object(2)
memory usage: 23.8+ MB


# List of preprocessing tasks
 - Lowercase the english letters
 - Check and remove null rows
 - CHeck if hindi is in english data
 - Use Regex to remove characters like @, #, $, %.
 - convert all the hindi numerals to integers
 - Unicode Normalization
 - 

In [179]:
# convert all the characters of english to lowercase
data["english"] = data["english"].apply(lambda x: str(x).lower())

In [180]:
# Check if any null values
data.isna().sum()   # no row is empty

hindi      6057
english       0
dtype: int64

In [181]:
# Dropping the NA values
data.dropna(inplace=True)
data.isna().sum()

hindi      0
english    0
dtype: int64

In [182]:
data["english"]   # this shows that there are hindi characters in english column, we need to drop those rows

0             give your application an accessibility workout
1                          accerciser accessibility explorer
2             the default plugin layout for the bottom panel
3                the default plugin layout for the top panel
4             a list of plugins that are disabled by default
                                 ...                        
1561835                  शपथ लेने/प्रतिज्ञान करने वाले सदस्य
1561836    स्पष्टीकरण.–जहां इस उपधारा के अधीन हानि और लाभ...
1561837    है। i note that this is a landmark meeting – n...
1561838    है। in the presentations that they made before...
1561839    ्त है। issues such as food and water security;...
Name: english, Length: 1555784, dtype: object

In [183]:
# regex checking of characters [\u0900-\u097F]+ will tell us all the rows in english column that have hindi character
# return np.nan for rows that contain hindi, and drop them
# also .apply doesn't do permanent changes, good for looking

def preprocess_english(line):
     # if it contains letters of hindi
    if re.search("[\u0900-\u097F]+", line):  
        return np.nan
    else:
        line = re.sub("[^\w\s\.\?!\।]", "", line)   # remove unnecessary puncuations
        line = unicodedata.normalize("NFKC", line)   # normalize unicode
        line = re.sub(r"\s+", " ", line).strip()   # remove extra whitespaces
        return line
        
data["english"] = data["english"].apply(preprocess_english)
data.dropna(inplace=True)

In [184]:
data   # as we can see the hindi in english column has been removed

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,accerciser accessibility explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,the default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,the default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,a list of plugins that are disabled by default
...,...,...
1561828,जोनलः रू. 0.50 लाख के प्रत्येक वार्षिक पुरस्का...,zonal eleven annual awards of 0.50 lakh each e...
1561829,इस्लाम से पहले करोड़ों लोगों के इस पारसी धर्म ...,zoroastrianism which was once the religion of ...
1561830,"(य) ""प्रतिभूतिकरण"" से किसी प्रतिभूतिकरण कंपनी ...",z securitisation means acquisition of financia...
1561831,राष्ट्रपति जी ने कहा कि जुबिन मेहता संगीत को श...,zubin mehta has engaged in untiring efforts to...


In [185]:
def preprocess_hindi(line):
    
    # Unicode Normalisation
    line = unicodedata.normalize("NFKC", line)

    # removing the unnecessary punctuations
    # This pattern says: 
    # "Find everything that is NOT a space, NOT a word character, 
    # AND NOT a character in the Devanagari Unicode range (U+0900 to U+097F)"
    line = re.sub(r'[^\s\w\u0900-\u097F\.\?!\।]', '', line)
    
    # add space before ending puncuations to make them seperate tokens
    # Space out ?, !, and Hindi Purna Viram
    line = re.sub(r"([\?!\।])", r" \1 ", line)
    # Space out dots only if they aren't decimals
    line = re.sub(r"(?<!\d)\.(?!\d)", r" . ", line)
    
    # remove extra whitespaces
    line = re.sub(r"\s+", " ", line).strip()
    
    # Start and end block for decoder with space (for seperate tokenisation)
    line = "<START> " + line + " <END>"

    return line
    
data["hindi"] = data["hindi"].apply(preprocess_hindi)
data["hindi"]

0          <START> अपने अनुप्रयोग को पहुंचनीयता व्यायाम क...
1              <START> एक्सेर्साइसर पहुंचनीयता अन्वेषक <END>
2          <START> निचले पटल के लिए डिफोल्ट प्लगइन खाका <...
3          <START> ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका <END>
4          <START> उन प्लगइनों की सूची जिन्हें डिफोल्ट रू...
                                 ...                        
1561828    <START> जोनलः रू . 0.50 लाख के प्रत्येक वार्षि...
1561829    <START> इस्लाम से पहले करोड़ों लोगों के इस पार...
1561830    <START> य प्रतिभूतिकरण से किसी प्रतिभूतिकरण कं...
1561831    <START> राष्ट्रपति जी ने कहा कि जुबिन मेहता सं...
1561832    <START> जुबिन मेहता के संगीत में सीमाओं के आरप...
Name: hindi, Length: 1553212, dtype: object

In [186]:
data

,hindi,english
0,<START> अपने अनुप्रयोग को पहुंचनीयता व्यायाम क...,give your application an accessibility workout
1,<START> एक्सेर्साइसर पहुंचनीयता अन्वेषक <END>,accerciser accessibility explorer
2,<START> निचले पटल के लिए डिफोल्ट प्लगइन खाका <...,the default plugin layout for the bottom panel
3,<START> ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका <END>,the default plugin layout for the top panel
4,<START> उन प्लगइनों की सूची जिन्हें डिफोल्ट रू...,a list of plugins that are disabled by default
...,...,...
1561828,<START> जोनलः रू . 0.50 लाख के प्रत्येक वार्षि...,zonal eleven annual awards of 0.50 lakh each e...
1561829,<START> इस्लाम से पहले करोड़ों लोगों के इस पार...,zoroastrianism which was once the religion of ...
1561830,<START> य प्रतिभूतिकरण से किसी प्रतिभूतिकरण कं...,z securitisation means acquisition of financia...
1561831,<START> राष्ट्रपति जी ने कहा कि जुबिन मेहता सं...,zubin mehta has engaged in untiring efforts to...
